# USA Electricity Generating Exploratory Analysis

In [9]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_file, show, output_notebook
from bokeh.palettes import Spectral6
from bokeh.transform import dodge, factor_cmap
from bokeh.models import ColumnDataSource, FactorRange
output_notebook()
df_2018 = pd.read_csv("raw_data/y2018.csv")


## In 2018
# What is the breakdown of capacity by technology
# What is the breakdown of usable electricty by technology
# What is the breakdown of capacity by renewables, fossil fuel, nuclear
# What is the breakdown of capacity by technology by state
# Geolocation of plants by different technologies - nuclear, coal, NG, hydro
## Larger questions
# How does energy makeup change throughout the years


Loading BokehJS ...

In [3]:
print(df_2018.head(10), "\n")

print(f"Electricity generating technologies in this dataset: {df_2018.Technology.unique()}")

print(f"Number of power plants in this dataset: {df_2018.Plant_ID.nunique()}")

   Nameplate     Technology   Latitude  Longitude State  Plant_ID
0       53.9  Hydroelectric  33.458665 -87.356823    AL         2
1      153.1    Natural Gas  31.006900 -88.010300    AL         3
2      153.1    Natural Gas  31.006900 -88.010300    AL         3
3      403.7           Coal  31.006900 -88.010300    AL         3
4      788.8           Coal  31.006900 -88.010300    AL         3
5      170.1    Natural Gas  31.006900 -88.010300    AL         3
6      170.1    Natural Gas  31.006900 -88.010300    AL         3
7      195.2    Natural Gas  31.006900 -88.010300    AL         3
8      170.1    Natural Gas  31.006900 -88.010300    AL         3
9      170.1    Natural Gas  31.006900 -88.010300    AL         3 

Electricity generating technologies in this dataset: ['Hydroelectric' 'Natural Gas' 'Coal' 'Other' 'Nuclear' 'Wind' 'Solar']
Number of power plants in this dataset: 9062


Columns include the nameplate capacity, the generating technology, the location, and the Plant_ID.

######  Nameplate Capacity
The nameplate capacity is the "rated" capacity of the power plant, but it's important to note that this is just a theoretical maximum. For example, a solar plant could be rated for 10 MW in sunny, perfect conditions, but in reality, it might only generate a fraction of that because the sun doesn't always shine. There is a name for this: [Capacity Factor](https://en.wikipedia.org/wiki/Capacity_factor). 

Below are capacity factors taken from Wikipedia for 2018. Natural gas is assumed to mostly be combined cycle (Cogen) and other is assumed by be biomass, geothermal, etc. Solar is assumed to be photovoltaic (PV)

###### Repeat Columns
Notice rows 2-9. Looks like a repeat right (especially 5 and 6)? There are numerous rows that look like duplicates but are actually different modules of one power plant. For example, it could be two separate nuclear reactors at one site or two cogeneration trains at a natural gas plant. For the purposes of this analysis, it will be best to combine these with a sum these by technology to get the sense of the electricity generating capacity at a facility

In [6]:
capacity_factors = {"Hydroelectric": .428, 
                        "Wind": .374, 
                        "Solar": .261, 
                        "Coal": .54, 
                        "Natural Gas":.5,
                        "Nuclear": .887, 
                        "Other": .5}

electricity_classification = {"Hydroelectric": "Renewable", 
                              "Wind": "Renewable", 
                              "Solar": "Renewable", 
                              "Nuclear": "Renewable", 
                              "Other": "Renewable", 
                              "Coal": "Fossil Fuel",
                              "Natural Gas": "Fossil Fuel"}

# Creating a function in case we would like to repeat this data cleansing
def cleanse_df(df, capacity_factors=capacity_factors, electricity_classification=electricity_classification):
    df = df.groupby(["Plant_ID", "Technology", "Latitude", "Longitude", "State"], as_index=False).sum()
    df["Classification"] = df["Technology"].map(electricity_classification)
    df["Electricity Output (MW)"] = np.round(df["Nameplate"] * df["Technology"].map(capacity_factors),1)
    return df

df_2018_clean = cleanse_df(df_2018)

# If a powerplant has two modes of generation (eg Coal and NG), the Plant_ID will still be repeated

In [5]:
largest_gen_df_2018 = df_2018_clean.sort_values("Electricity Output (MW)", ascending=False) \
                              .drop(columns=["Longitude", "Latitude"])
with pd.option_context('expand_frame_repr', False):
    print(largest_gen_df_2018.head(10))

      Plant_ID     Technology State  Nameplate Classification  Electricity Output (MW)
2007      6008        Nuclear    AZ     4209.6      Renewable                   3733.9
25          46        Nuclear    AL     3494.0      Renewable                   3099.2
2126      6163  Hydroelectric    WA     6809.0      Renewable                   2914.3
1627      3166        Nuclear    PA     2876.4      Renewable                   2551.4
2170      6251        Nuclear    TX     2708.6      Renewable                   2402.5
1647      3265        Nuclear    SC     2666.7      Renewable                   2365.4
2560      7722        Nuclear    TN     2539.8      Renewable                   2252.8
2084      6103        Nuclear    PA     2532.0      Renewable                   2245.9
2021      6023        Nuclear    IL     2449.8      Renewable                   2173.0
2020      6022        Nuclear    IL     2449.8      Renewable                   2173.0


It's no surprise that the 9/10 of the largest generators are nuclear plants. 

<img src="images/Grand_Coulee_Dam.jpg" width="340" alt="Grand Coulee Dam" align="left"><br clear="left">

The hydroelectric plant in Washington is the Grand Coulee Dam (image source: Wikipedia)


In [7]:
# Take advantage of Bokeh's ColumnDataSource class
drop_cols = ["Plant_ID", "State", "Longitude", "Latitude"]
# Convert to gigawatts
for c in ["Electricity Output (MW)", "Nameplate"]:
    df_2018_clean[c] = df_2018_clean[c]/10**3
    
tech_group = df_2018_clean.drop(columns=drop_cols) \
                          .groupby("Technology") \
                          .agg({"Electricity Output (MW)": 'sum', "Nameplate": 'sum'}) \
                          .sort_values(by="Nameplate", ascending=False) \
                          .reset_index()

source = ColumnDataSource(tech_group)


p = figure(plot_height=400, x_range=tech_group["Technology"], title="Electricity Output by Technology",
           toolbar_location=None,  y_axis_label = "Gigawatts")
p.vbar(x="Technology", top="Electricity Output (MW)", width=.9, source=source, color="#e6e600")
p.y_range.start = 0
p.xgrid.grid_line_color = None
show(p)

In [41]:
df_bars = df_2018_clean.copy()
df_bars = df_bars.groupby("Technology").sum().reset_index().sort_values("Nameplate", ascending=True)

technologies = list(df_bars["Technology"])
categories = ["Electricity Output", "Nameplate"]
output = list(df_bars["Electricity Output (MW)"])
nameplate = list(df_bars["Nameplate"])

x = [(technology, category) for technology in technologies for category in categories]
counts = sum(zip(output, nameplate), ())
source = ColumnDataSource(data=dict(x=x, counts=counts))


f = figure(y_range=FactorRange(*x), plot_width=800, title="Nameplate & Output", x_axis_label = "Gigawatts")
f.hbar(y='x', right='counts', width=.9,  color="#e6e600", source=source,
      fill_color=factor_cmap('x', palette=['firebrick', 'olive'], factors=categories, start=1, end=2))
f.ygrid.grid_line_color = None
f.x_range.start = 0
f.x_range.range_padding = 0.1
show(f)